In [25]:
#miseq mini reference generator
#first read files of just between the genome primers then read both LR and RL insert direction files
#L02: accagacccgcgagcattaattcttgcctcca
#L09: tggtggcggtggtgggagctattctcgttctg
#L14: gaaaacacctgatatgaaaggcaatgccacca
#L21: agctgcaacaatgttgaaaatgccagccaact
import os
import subprocess
from Bio.Seq import Seq
path = 'sequencing-refs/L02/'
genome_file = open(path + "L02.fa", "r")
first = genome_file.readline()
genome_str = genome_file.read()
genome_file.close()
genome_str = genome_str.replace('\n','')

RL_file = open(path + "../R-sc-GFP-L.fa", "r")
first = RL_file.readline()
RL_str = RL_file.read()
RL_file.close()
RL_str = RL_str.replace('\n','')


LR_file = open(path + "../L-GFP-sc-R.fa", "r")
first = LR_file.readline()
LR_str = LR_file.read()
LR_file.close()
LR_str = LR_str.replace('\n','')

In [27]:
crRNA = 'accagacccgcgagcattaattcttgcctcca'
crRNA_ind = genome_str.index(crRNA) + 32
crRNA_ind

153

In [40]:
#write on this fasta file
f = open(path + "l21-mini-references.fa", "a")

In [28]:
# start at 40th index after crRNA_ind and end at 61th
# f.write()
# f.close()
counter = 40
seed = []
headers = []
seed_window = 16
while counter <= 61:
    # f.write(f'>{counter}bp-RL-direction\n')
    # f.write(genome_str[:crRNA_ind] + genome_str[crRNA_ind: crRNA_ind + counter] + RL_str + genome_str[crRNA_ind + counter-5:crRNA_ind + counter] + genome_str[crRNA_ind+counter:] + '\n')
    # f.write(f'>{counter}bp-LR-direction\n')
    # f.write(genome_str[:crRNA_ind] + genome_str[crRNA_ind: crRNA_ind + counter] + LR_str + genome_str[crRNA_ind + counter-5:crRNA_ind + counter] + genome_str[crRNA_ind+counter:] + '\n')
    seed_1 = genome_str[crRNA_ind + counter - int(seed_window/2)-1: crRNA_ind + counter-1].upper() + RL_str[:8].upper()
    seed_2 = genome_str[crRNA_ind + counter - int(seed_window/2)-1: crRNA_ind + counter-1].upper() + LR_str[:8].upper()
    seed.append(seed_1)
    seed.append(seed_2)
    headers.append(f'>{counter}bp-RL-direction')
    headers.append(f'>{counter}bp-LR-direction')
    counter += 1

In [29]:
seed[:-2]
headers[:-2]
seed_with_headers = {}
header_align_count = {}
for i in range(len(seed)):
    seed_with_headers.update({headers[i]: seed[i]})
    header_align_count.update({headers[i]: 0})
with open(path+"L02.fastq") as file:
    lines = [line.rstrip() for line in file]
line_index = [x for x in range(len(lines)) if "@" in lines[x]]

In [32]:
reads = []
for i in line_index:
    reads.append(lines[i+1])

In [33]:
with open(path + 'reads.txt', 'w') as f:
    for x in reads:
        f.write("%s\n" % x)
# for x in seed_with_headers.values():
#     if any(x in y for y in reads):
#         print('yes')
#         key = seed_with_headers.keys()[list(seed_with_headers.values()).index(x)]
#         print(key)
#         header_align_count[seed_with_headers.getkey(x)] += 1

In [34]:
reads_file = path + 'reads.txt'
key_list = list(seed_with_headers.keys())
val_list = list(seed_with_headers.values())
for x in seed_with_headers.values():
    output53 = path + 'output53.txt'
    output35 = path + 'output35.txt'
    seq = Seq(x)
    os.system(f'grep {x} {reads_file} | wc -l >> {output53}')
    os.system(f'grep {seq.reverse_complement()} {reads_file} | wc -l >> {output35}')

In [2]:
path = 'sequencing-refs/L21/'
output53 = path + 'output53.txt'
output35 = path + 'output35.txt'
o53 = open(output53, 'r')
o35 = open(output53, 'r')
o53_lines = [line.rstrip() for line in o53]
o35_lines = [line.rstrip() for line in o35]
number_of_reads_separated_by_direction = []
number_of_reads_no_direction = []
for i in range(len(o53_lines)):
    number_of_reads_separated_by_direction.append(int(o53_lines[i]) + int(o35_lines[i]))
    number_of_reads_no_direction = [sum(number_of_reads_separated_by_direction[i:i+2]) for i in range(0, len(number_of_reads_separated_by_direction), 2)]
len(o53_lines)

44

In [4]:
import numpy as np
import pandas as pd

plot_dict = {}
positions = []
read_fraction = []
total_reads = sum(number_of_reads_no_direction)
df = pd.DataFrame(columns=['position', 'read_fraction', 'direction'])

# no direction
# df_no_direction = pd.DataFrame(columns=['position', 'read_fraction'])
# for n, i in enumerate(number_of_reads_no_direction):
#     plot_dict.update({i/total_reads : n + 40})
#     positions.append(n + 40)
#     read_fraction.append(i/total_reads)
# df_no_direction["position"] = positions
# df_no_direction["read_fraction"] = read_fraction

direction = []
plot_dict_RL = {}
plot_dict_LR = {}
total_reads_RL = sum(number_of_reads_separated_by_direction[::2])
total_reads_LR = sum(number_of_reads_separated_by_direction[1::2])
position_counter = 40
#separated by direction
for n, i in enumerate(number_of_reads_separated_by_direction):
    if not n%2:
        plot_dict_RL.update({i/total_reads_RL : n + 40})
        read_fraction.append(i/total_reads_RL)
        positions.append(n + position_counter)
        direction.append('RL')
        position_counter = position_counter - 1
    else:
        plot_dict_LR.update({i/total_reads_LR : n + 39})
        read_fraction.append(i/total_reads_LR)
        positions.append(n + position_counter)
        direction.append('LR')
df["position"] = positions
df["read_fraction"] = read_fraction
df["direction"] = direction
df["loci"] = 'L21'
#df.to_csv(path+'read_fractions.csv')
df.to_csv('sequencing-refs/L21/'+'read_fractions.csv', mode='a', index=False, header=False)
df

,position,read_fraction,direction,loci
0,40,0.000000,RL,L21
1,40,0.000000,LR,L21
2,41,0.000000,RL,L21
3,41,0.000000,LR,L21
4,42,0.000000,RL,L21
5,42,0.000000,LR,L21
6,43,0.000000,RL,L21
7,43,0.000000,LR,L21
8,44,0.000000,RL,L21
9,44,0.000000,LR,L21


In [50]:
### STAT = DENSITY for plotting
hist_positions = []
hist_direction = []
for i in range(len(positions)):
    if direction[i] == 'RL':
        hist_positions += [positions[i]]*number_of_reads_separated_by_direction[i]
        hist_direction += ['RL']*number_of_reads_separated_by_direction[i]
    else:
        hist_positions += [positions[i]]*number_of_reads_separated_by_direction[i]
        hist_direction += ['LR']*number_of_reads_separated_by_direction[i]
df_hist = pd.DataFrame(columns=['position', 'direction'])
df_hist["position"] = hist_positions
df_hist["direction"] = hist_direction
df_hist.to_csv(path + 'L21-hist')